# Calculadora detallada de sou net

En aquest document podràs calcular i visualitzar clarament quins impostos pagues, quin sou et queda mensual i quan t'incrementa el sou mensual quan et pugen el sou.

**Instruccions d'us:** Executa totes les cel·les en ordre (o premer "Run all"), a l'arribar la cel·la final, omple els camps i calcula el teu sou net.


In [51]:
#@title Imports & Utils

from dataclasses import dataclass, field
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import List, Tuple, Optional, Dict

# Increse precission
getcontext().prec = 28

# Forma d'arrodoniment per defecte: arrodonir a 2 decimals, meitat endavant (banker's? no: half-up)
def round_euro(x: Decimal) -> Decimal:
    return x.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)


In [52]:
#@title Seguretat Social (valors públics 2025)
#@markdown Editar la cel·la per modificar: Bases màximes i mínimes de cotització per grup i tipus de cotització

# -------------------------
# Seguretat Social 2025
# -------------------------

# Bases màximes
SS_BASE_MAX_MONTHLY = Decimal("4909.50")
SS_BASE_MAX_DAILY = Decimal("163.65")

# Bases mínimes per grup
SS_BASE_MIN_BY_GROUP = {
    "Sou mensual; adult; Enginyeres i llicenciades universitàries": Decimal("1929.00"),
    "Sou mensual; adult; Enginyers tècnics, perits i ajudants titulats": Decimal("1599.60"),
    "Sou mensual; adult; Caps administratius i de taller": Decimal("1391.70"),
    "Sou mensual; adult; altres": Decimal("1381.20"),
    "Base diaria; adult": Decimal("46.04"),
    "Menor d'edat": Decimal("46.04"),
}

# Tipus de cotització (treballador)
DEFAULT_SS_RATES = {
    "contingencies_common_worker": Decimal("0.0470"),
    "unemployment_worker_indefinite": Decimal("0.0155"),
    "unemployment_worker_temporary": Decimal("0.0160"),
    "training_worker": Decimal("0.0010"),
    "mei_worker": Decimal("0.0013"),
}


In [53]:
#@title Mínim personal / familiar (valors públics 2025)
#@markdown Editar la cel·la per modificar: Mínim contribuent general, i els seus increments per edat, fills o discapacitats

MINIMO_CONTRIBUYENTE = Decimal("5550.00")  # contribuent general

# Increment per edat
AGE_ADJUSTMENTS = [
    (65, Decimal("1150")),  # +1150 € si >65
    (75, Decimal("2550")),  # +2550 € si >75
]

CHILDREN_UNDER_25_ADJUSTMENT = [
    (1, Decimal("2400")),  # primer fill
    (2, Decimal("2700")),  # segon fill
    (3, Decimal("4000")),  # tercer i següents
]
ASCENDENTS_ADJUSTMENT = [Decimal("1150"), Decimal("1400"), Decimal("1500")]


DISABILITY_SELF_ADJUSTMENTS = [
    (33, 3000),
    (65, 9000),
]
DISABILITY_RELATIVES_ADJUSTMENTS = [
    (33, 1000),
    (65, 2000),
]



In [61]:
#@title Escala IRPF estatal i autonòmica (valors públics 2025)
#@markdown Editar la cel·la per modificar les escales d'IRPF estatal i autonòmiques

IRPF_SCALE_ESTATAL = [
            (0, 12450, Decimal("0.095")),
            (12450, 20200, Decimal("0.12")),
            (20200, 35200, Decimal("0.15")),
            (35200, 60000, Decimal("0.185")),
            (60000, 300000, Decimal("0.225")),
            (300000, None, Decimal("0.245"))
        ]

### Comunitats autònomes

IRPF_SCALE_CATALUNYA = [
            (0, 12450, Decimal("0.12")),
            (12450, 17707.20, Decimal("0.12")),
            (17707.20, 21000, Decimal("0.14")),
            (21000, 33007.20, Decimal("0.15")),
            (33007.20, 53407.20, Decimal("0.188")),
            (53407.20, 90000, Decimal("0.215")),
            (90000, None, Decimal("0.235"))
        ]


###########################
# TODO: Augmentar CCOO
###########################


In [55]:
#@title Estructures de dades

@dataclass
class FamilySituation:
    age: int = 30
    children_ages: List[int] = field(default_factory=list)
    children_disabilities: List[int] = field(default_factory=list)
    ascendents_dependent: int = 0
    disability_percent_self: int = 0
    disability_percent_relatives: int = 0

    def minimo_personal_familiar(self) -> Decimal:
        """
        Calcula el mínim personal i familiar segons AEAT 2025.
        Tot el que pot canviar està separat en constants globals.
        """
        minimo = MINIMO_CONTRIBUYENTE

        # 1) Ajust per edat del contribuent
        for age_limit, inc in AGE_ADJUSTMENTS:
            if self.age > age_limit:
                minimo += inc

        # 2) Ajust per fills
        for i, child_age in enumerate(self.children_ages):
            if child_age <= 25:
                child_order = i + 1
                increment = None
                for order_limit, value in CHILDREN_UNDER_25_ADJUSTMENT:
                    if child_order <= order_limit:
                        increment = value
                        break
                if increment is None:
                    increment = CHILDREN_UNDER_25_ADJUSTMENT[-1][1]
                minimo += increment

        # 3) Ascendents dependents
        for i in range(self.ascendents_dependent):
            if i < len(ASCENDENTS_ADJUSTMENT):
                minimo += ASCENDENTS_ADJUSTMENT[i]
            else:
                minimo += ASCENDENTS_ADJUSTMENT[-1]

        # 4) Discapacitat del contribuent
        for perc_limit, inc in DISABILITY_SELF_ADJUSTMENTS:
            if self.disability_percent_self >= perc_limit:
                minimo += inc

        # 5) Discapacitat de familiars dependents
        for perc_limit, inc in DISABILITY_RELATIVES_ADJUSTMENTS:
            if self.disability_percent_relatives >= perc_limit:
                minimo += inc

        return minimo




@dataclass
class IRPFScale:
    """
    Representa l'escala d'IRPF com a llista de trams:
    List[ (from_exclusive_or_inclusive, to_inclusive_or_None, rate_decimal) ]
    """
    brackets: List[Tuple[Decimal, Optional[Decimal], Decimal]] = field(default_factory=list)

    def tax_on_base(self, base: Decimal) -> Decimal:
        """
        Aplica l'escala progressiva i retorna l'import total d'impost (sense deduccions).
        """
        tax = Decimal("0")
        for low, high, rate in self.brackets:
            low = Decimal(low)
            high = Decimal(high) if high is not None else None
            if base <= low:
                break
            upper = base if (high is None or base < high) else high
            taxable = upper - low
            if taxable <= 0:
                continue
            tax += taxable * rate
        return tax

    @classmethod
    def combined_scale(cls, regional_scale: List[Tuple[Decimal, Optional[Decimal], Decimal]],
                       state_scale: List[Tuple[Decimal, Optional[Decimal], Decimal]]) -> "IRPFScale":
        """
        Combina escala autonòmica + estatal en una sola escala progressiva.
        """
        combined_brackets = []
        for i in range(max(len(regional_scale), len(state_scale))):
            r_low, r_high, r_rate = regional_scale[i] if i < len(regional_scale) else (None, None, 0)
            s_low, s_high, s_rate = state_scale[i] if i < len(state_scale) else (None, None, 0)
            # El tram finalitza quan acaba qualsevol de les dues escales, o None si és infinit
            low = Decimal(r_low) if r_low is not None else Decimal(s_low)
            high_candidates = [h for h in [r_high, s_high] if h is not None]
            high = Decimal(min(high_candidates)) if high_candidates else None
            combined_brackets.append((low, high, Decimal(r_rate) + Decimal(s_rate)))
        return cls(combined_brackets)


# -------------------------
# Funcions de càlcul de SS
# -------------------------
def apply_base_limits(base: Decimal,
                      base_min: Decimal,
                      base_max: Decimal,
                      is_daily: bool = False,
                      days_in_month: int = 30) -> Decimal:
    """
    Ajusta la base entre mínim i màxim legals.
    Si és base diària, aplica mínim/màxim diari i després multiplica pels dies del mes.
    """
    if is_daily:
        base_diari = base / Decimal(days_in_month)
        base_diari_ajustat = max(base_min, min(base_diari, base_max))
        return base_diari_ajustat * Decimal(days_in_month)
    else:
        return max(base_min, min(base, base_max))


def calculate_social_security_worker(monthly_base: Decimal,
                                      ss_rates: Dict[str, Decimal] = DEFAULT_SS_RATES,
                                      contract_type: str = "indefinite",
                                      base_min: Decimal = SS_BASE_MIN_GENERIC_2025,
                                      base_max: Decimal = SS_BASE_MAX_2025,
                                      is_daily: bool = False,
                                      days_in_month: int = 30,
                                      rounding: bool = True) -> Decimal:
    # Ajust base segons mensual/diari
    base = apply_base_limits(monthly_base, base_min, base_max, is_daily=is_daily, days_in_month=days_in_month)

    # Tipus d’atur segons contracte
    t_des = ss_rates["unemployment_worker_indefinite"] if contract_type == "indefinite" else ss_rates["unemployment_worker_temporary"]

    total_rate = ss_rates["contingencies_common_worker"] + t_des + ss_rates["training_worker"] + ss_rates["mei_worker"]
    cotitzacio = base * total_rate
    if rounding:
        cotitzacio = round_euro(cotitzacio)
    return cotitzacio


# -------------------------
# Funcions IRPF (base anual i retenció)
# -------------------------
def calculate_base_imposable_irpf(annual_gross: Decimal,
                                  annual_employee_ss: Decimal,
                                  family: FamilySituation,
                                  other_deductions: Decimal = Decimal("0")) -> Decimal:
    """
    Base imposable anual simplificada per l'IRPF:
    base_imposable = brut_anual - cotitzacions_anuals - minim_personal_i_familiar - altres deduccions
    Observació: l'AEAT aplica criteris addicionals (liquidable, etc.); funció simplificada.
    """
    minimo_pf = family.minimo_personal_familiar()
    base = annual_gross - annual_employee_ss - minimo_pf - other_deductions
    if base < 0:
        base = Decimal("0")
    return base

def calculate_irpf_annual_from_scale(base_imposable: Decimal,
                                     scale: IRPFScale,
                                     rounding: bool = True) -> Decimal:
    """
    Calcula l'IRPF anual a partir d'una escala (estat + autonòmica combinada).
    Aquesta funció no implementa l'algoritme complet AEAT per a retencions a practicar,
    però sí fa el càlcul progressiu bàsic. Per a coincidència amb calculadora AEAT/BBVA,
    s'hauria d'implementar l'algoritme de retencions AEAT (pdf tècnic).
    """
    tax = Decimal("0")
    # Implementació estàndard progressiva per trams:
    for lower, upper, rate in scale.brackets:
        lower = Decimal(lower)
        upper = Decimal(upper) if upper is not None else None
        if base_imposable <= lower:
            break
        if upper is None or base_imposable < upper:
            taxable = base_imposable - lower
        else:
            taxable = upper - lower
        tax += taxable * Decimal(rate)
    if rounding:
        tax = round_euro(tax)
    return tax


In [115]:
#@title Funció principal amb gràfics i sortida clara
import matplotlib.pyplot as plt
from decimal import Decimal
from typing import Dict

def compute_net_pay(
    basic_annual_gross: Decimal,
    n_pagues: int = 12,
    pagues_prorratejades: bool = True,
    retribucio_en_especie_ann: Decimal = Decimal("0"),
    grup_cotització: str = "Sou mensual; adult; Enginyeres i llicenciades universitàries",
    contract_type: str = "indefinite",
    family: FamilySituation = FamilySituation(),
    region: str = "catalunya",
    ss_rates: Dict[str, Decimal] = DEFAULT_SS_RATES,
    ss_base_min: Decimal = SS_BASE_MIN_GENERIC_2025,
    ss_base_max: Decimal = SS_BASE_MAX_2025,
    other_annual_deductions: Decimal = Decimal("0"),
    rounding: bool = True
) -> Dict[str, Decimal]:

    assert region.lower()=="catalunya", "De moment només la regió de Catalunya està disponible"

    # Escala IRPF combinada
    if region.lower() == "catalunya":
        irpf_scale = IRPFScale.combined_scale(IRPF_SCALE_CATALUNYA, IRPF_SCALE_ESTATAL)

    gross_including_benefits = basic_annual_gross + retribucio_en_especie_ann
    monthly_base = gross_including_benefits / Decimal(n_pagues if pagues_prorratejades else 12)

    is_daily = grup_cotització in ["Base diaria; adult", "Menor d'edat"]
    cotitzacions_mensuals = calculate_social_security_worker(
        monthly_base,
        ss_rates,
        contract_type,
        ss_base_min,
        ss_base_max if not is_daily else SS_BASE_MAX_DAILY_2025,
        is_daily=is_daily,
        days_in_month=30,
        rounding=rounding
    )
    cotitzacions_anuals = cotitzacions_mensuals * Decimal(n_pagues)

    base_imposable = calculate_base_imposable_irpf(
        gross_including_benefits, cotitzacions_anuals, family, other_deductions=other_annual_deductions
    )

    irpf_anual = irpf_scale.tax_on_base(base_imposable)
    if rounding:
        irpf_anual = round_euro(irpf_anual)

    irpf_per_paga = irpf_anual / Decimal(n_pagues)
    if rounding:
        irpf_per_paga = round_euro(irpf_per_paga)

    gross_per_paga = gross_including_benefits / Decimal(n_pagues)
    net_per_paga = gross_per_paga - cotitzacions_mensuals - irpf_per_paga
    if rounding:
        net_per_paga = round_euro(net_per_paga)

    net_monthly_equivalent = net_per_paga * Decimal(n_pagues) / Decimal(12) if n_pagues != 12 else net_per_paga

    # ---------------------------
    # Gràfic 1: Pie chart dels impostos
    # ---------------------------
    taxes_labels = ["Cotitzacions a la Seguretat Social (empleat)", "Impost sobre la Renda de les Persones Físiques"]
    taxes_values = [float(cotitzacions_anuals), float(irpf_anual)]

    plt.figure(figsize=(6,6))
    plt.pie(taxes_values, labels=taxes_labels, autopct='%1.1f%%', startangle=90, colors=["#66b3ff","#ff9999"])
    plt.title("Distribució dels impostos i cotitzacions sobre el sou anual")
    plt.show()

    # ---------------------------
    # Gràfic 2: Bar chart cada 1000€
    # ---------------------------
    n_blocks = int(basic_annual_gross // 1000) + 1
    block_values = []
    for i in range(n_blocks):
        gross_block = min(1000, float(basic_annual_gross - i*1000))
        # Proporció taxes i sou net per bloc
        ss_block = float(cotitzacions_anuals) * (gross_block / float(basic_annual_gross))
        irpf_block = float(irpf_anual) * (gross_block / float(basic_annual_gross))
        net_block = gross_block - ss_block - irpf_block
        block_values.append([net_block, ss_block + irpf_block])

    import numpy as np
    block_values = np.array(block_values)
    plt.figure(figsize=(10,5))
    plt.bar(range(n_blocks), block_values[:,0], label="Sou net per bloc de 1000€", color="#99ff99")
    plt.bar(range(n_blocks), block_values[:,1], bottom=block_values[:,0], label="Impostos i cotitzacions per bloc de 1000€", color="#ff6666")
    plt.xlabel("Blocs de 1000€ de sou brut anual")
    plt.ylabel("Euros")
    plt.title("Distribució de sou net i impostos per cada 1000€ de sou brut")
    plt.legend()
    plt.show()

    # ---------------------------
    # Sortida clara en català
    # ---------------------------
    print("=== RESUM DEL SOU ANUAL I IMPOSTOS ===")
    print(f"SOU BRUT ANUAL TOTAL (incloent retribució en espècie): {gross_including_benefits:.2f} €")
    print(f"Base mensual per cotitzacions a la Seguretat Social: {monthly_base:.2f} €")
    print(f"Cotitzacions anuals a la Seguretat Social (empleat): {cotitzacions_anuals:.2f} €")
    print(f"Base imposable per l'Impost sobre la Renda: {base_imposable:.2f} €")
    print(f"IRPF anual: {irpf_anual:.2f} €")
    print(f"SOU NET PER PAGA: {net_per_paga:.2f} €")
    print(f"SOU NET MENSUAL EQUIVALENT: {net_monthly_equivalent:.2f} €")
    print("=======================================")

    return {
        "sou_brut_anual_total": gross_including_benefits,
        "base_mensual_per_ss": monthly_base,
        "cotitzacions_anuals_empleat": cotitzacions_anuals,
        "base_imposable_irpf": base_imposable,
        "irpf_anual": irpf_anual,
        "sou_net_per_paga": net_per_paga,
        "sou_net_mensual_equivalent": net_monthly_equivalent,
    }


In [118]:
from re import M
from IPython.display import display, Markdown
import ipywidgets as widgets
from decimal import Decimal

# -------------------------
# Títol general
# -------------------------
display(Markdown("## 🔹 Calculadora de sou net 2025"))
display(Markdown("Omple les dades a continuació. Prem el botó 'Calcular sou net' quan hagis acabat."))

# Layout general per a tots els widgets
desc_width = '200px'
widget_width = '600px'

# -------------------------
# Comunitat autònoma
# -------------------------
comunitats = [
    "Andalusia","Aragó","Astúries","Balears","Canàries","Cantàbria",
    "Castella-La Manxa","Castella i Lleó","Catalunya","Extremadura",
    "Galícia","La Rioja","Madrid","Murcia","Navarra","País Basc","València"
]
display(Markdown("### 🌍 Comunitat autònoma"))
display(Markdown("Selecciona la teva comunitat per aplicar l'IRPF correcte (estat + autonòmic)."))
display(Markdown("ALERTA! De moment només funciona Catalunya."))
ca_selector = widgets.Dropdown(
    options=comunitats,
    value="Catalunya",
    description="Comunitat:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(ca_selector)

# -------------------------
# Dades salarials
# -------------------------
display(Markdown("### 💰 Dades salarials"))

annual_gross_input = widgets.FloatText(
    value=25000,
    description='Sou brut anual (€):',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

n_pagues_input = widgets.IntSlider(
    value=14, min=12, max=15, step=1,
    description='Nombre de pagues:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(Markdown("Si les pagues extraordinàries estan repartides, no ho marquis si cobres les pagues extraordinàries per seperat"))

pagues_prorratejades_input = widgets.Checkbox(
    value=False,
    description='Pagues prorratejades',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(Markdown("Valors que reps de l'empresa no en efectiu. p.e cotxe, bons"))

retribucio_en_especie_ann_input = widgets.FloatText(
    value=0,
    description='Retribució en espècie (€):',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

grup_cotitzacio_input = widgets.Dropdown(
    options=[
        "Sou mensual; adult; Enginyeres i llicenciades universitàries",
        "Sou mensual; adult; Enginyers tècnics, perits i ajudants titulats",
        "Sou mensual; adult; Caps administratius i de taller",
        "Sou mensual; adult; altres",
        "Base diaria; adult",
        "Menor d'edat"
    ],
    value="Sou mensual; adult; Enginyeres i llicenciades universitàries",
    description="Grup cotització:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)


contract_type_input = widgets.Dropdown(
    options=[("Indefinit", "indefinite"), ("Temporal", "temporary")],
    value="indefinite",
    description="Tipus contracte:",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(Markdown("Qualsevol deducció addicional. p.e plans de pensions..."))

other_annual_deductions_input = widgets.FloatText(
    value=0,
    description="Altres deduccions (€):",
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(widgets.VBox([
    annual_gross_input,
    n_pagues_input,
    pagues_prorratejades_input,
    retribucio_en_especie_ann_input,
    grup_cotitzacio_input,
    contract_type_input,
    other_annual_deductions_input
]))

# -------------------------
# Situació familiar
# -------------------------
display(Markdown("### 👪 Situació familiar"))

age_input = widgets.IntSlider(
    value=30, min=16, max=100, step=1,
    description='Edat:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

ascendents_input = widgets.IntSlider(
    value=0, min=0, max=5, step=1,
    description='Ascendents dependents:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

disability_self_input = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='% Discapacitat pròpia:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

disability_relatives_input = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='% Discapacitat familiars:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)

display(widgets.VBox([
    age_input,
    ascendents_input,
    disability_self_input,
    disability_relatives_input
]))

# -------------------------
# Fills
# -------------------------
display(Markdown("### 🧒 Fills"))
display(Markdown("Introdueix l'edat dels fills seperat per comes. p.e (12,22,26)"))
children_ages_input = widgets.Text(
    value="",
    description='Edat fills:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)
display(children_ages_input)
display(Markdown("Introdueix el percentatge de discapacitat dels fills (en cas de tenir-ne) també separada per comes. p.e (33,0,90)"))

children_disabilities_input = widgets.Text(
    value="",
    description='% discapacitat fills:',
    layout=widgets.Layout(width=widget_width),
    style={'description_width': desc_width}
)


display(children_disabilities_input)

# -------------------------
# Botó i output
# -------------------------
button = widgets.Button(
    description="Calcular sou net",
    button_style='success',
    layout=widgets.Layout(width='200px')
)
output = widgets.Output()
display(button, output)


# -------------------------
# Funció que s'executa al pitjar el botó
# -------------------------
def on_button_click(b):
    with output:
        output.clear_output()
        # Conversió inputs
        gross = Decimal(str(annual_gross_input.value))
        n_pagues = int(n_pagues_input.value)
        pagues_prorratejades = bool(pagues_prorratejades_input.value)
        retribucio_en_especie_ann = Decimal(str(retribucio_en_especie_ann_input.value))
        age = int(age_input.value)
        ascendents = int(ascendents_input.value)
        disability_self = int(disability_self_input.value)
        disability_relatives = int(disability_relatives_input.value)
        grup_cotitzacio = grup_cotitzacio_input.value
        contract_type = contract_type_input.value
        other_deductions = Decimal(str(other_annual_deductions_input.value))

        # Fills
        try:
            children_ages = [int(x.strip()) for x in children_ages_input.value.split(",") if x.strip() != ""]
        except:
            children_ages = []
        try:
            children_disabilities = [int(x.strip()) for x in children_disabilities_input.value.split(",") if x.strip() != ""]
        except:
            children_disabilities = [0]*len(children_ages)

        # Crear objecte FamilySituation
        fam = FamilySituation(
            age=age,
            children_ages=children_ages,
            children_disabilities=children_disabilities,
            ascendents_dependent=ascendents,
            disability_percent_self=disability_self,
            disability_percent_relatives=disability_relatives
        )

        # Crida al compute_net_pay
        result = compute_net_pay(
            basic_annual_gross=gross,
            n_pagues=n_pagues,
            pagues_prorratejades=pagues_prorratejades,
            retribucio_en_especie_ann=retribucio_en_especie_ann,
            grup_cotització=grup_cotitzacio,
            contract_type=contract_type,
            family=fam,
            region=ca_selector.value,
            other_annual_deductions=other_deductions
        )



button.on_click(on_button_click)


## 🔹 Calculadora de sou net 2025

Omple les dades a continuació. Prem el botó 'Calcular sou net' quan hagis acabat.

### 🌍 Comunitat autònoma

Selecciona la teva comunitat per aplicar l'IRPF correcte (estat + autonòmic).

ALERTA! De moment només funciona Catalunya.

...

Text(value='', description='Edat fills:', layout=Layout(width='600px'), style=DescriptionStyle(description_wid…

Introdueix el percentatge de discapacitat dels fills (en cas de tenir-ne) també separada per comes. p.e (33,0,90)

Text(value='', description='% discapacitat fills:', layout=Layout(width='600px'), style=DescriptionStyle(descr…

Button(button_style='success', description='Calcular sou net', layout=Layout(width='200px'), style=ButtonStyle…

Output()